In [15]:
import multiprocessing

In [16]:
def singlerun(z=1):
    N=200
    a=0
    for i in range(N):
        for j in range(N):
            for k in range(N):
                a+=j*i*k
    return a+z


In [17]:
singlerun(),singlerun(2)

(7880599000001, 7880599000002)

In [18]:
simrun =10

kaizu's code simplified

In [19]:
jobs=[]
pip1s=[]

for i in range(simrun):
    def target_wrapper(f, end_send):
        def wf(*args, **_kwargs):
            end_send.send(f(*args, **_kwargs))
        return wf    
    pip1,pip2=multiprocessing.Pipe()
    pip1s.append(pip1)
    p=multiprocessing.Process(target=target_wrapper(singlerun,pip2),args=(i,))
    p.daemon=1
    jobs.append(p)
    p.start()
out = [i.recv() for i in pip1s]
for j in jobs:
    j.join()
    

In [20]:
out

[7880599000000,
 7880599000001,
 7880599000002,
 7880599000003,
 7880599000004,
 7880599000005,
 7880599000006,
 7880599000007,
 7880599000008,
 7880599000009]

kaizu's ecell4 code

In [21]:
def run(target, n=1, **kwargs):
    def target_wrapper(f, end_send):
        def wf(*args, **_kwargs):
            end_send.send(f(*args, **_kwargs))
        return wf

    processes = []
    end_recvs = []
    for J in range(n):
        end_recv, end_send = multiprocessing.Pipe(False)
        end_recvs.append(end_recv)
        p = multiprocessing.Process(
            target=target_wrapper(target, end_send),args=(J,))
        p.daemon=1
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

    retval = [end_recv.recv() for end_recv in end_recvs]
    return [retval[i: i + n] for i in range(0, len(retval), n)] 

In [22]:
run(singlerun,n=simrun)

[[7880599000000,
  7880599000001,
  7880599000002,
  7880599000003,
  7880599000004,
  7880599000005,
  7880599000006,
  7880599000007,
  7880599000008,
  7880599000009]]

using Pool to distribute jobs

In [23]:
pool_size=multiprocessing.cpu_count()
print pool_size
pool=multiprocessing.Pool(processes=pool_size)
pool_out=pool.map(singlerun,range(simrun))
pool.close()
pool.join()
pool_out[:5]

6


[7880599000000, 7880599000001, 7880599000002, 7880599000003, 7880599000004]

In [24]:
def runpool(func,n):
    pool_size=multiprocessing.cpu_count()
    pool=multiprocessing.Pool(processes=pool_size)
    pool_out=[pool.apply_async(func,args=(x,)) for x in range(n)] #async=run n receive result indpnt of sequence
    pool.close()
    pool.join()
    return [p.get() for p in pool_out]

In [25]:
runpool(singlerun,simrun)

[7880599000000,
 7880599000001,
 7880599000002,
 7880599000003,
 7880599000004,
 7880599000005,
 7880599000006,
 7880599000007,
 7880599000008,
 7880599000009]

In [26]:
%%timeit?